In [1]:
# import numpy as np
# from numpy import linalg as LA
# import torch; torch.set_default_dtype(torch.float64)
# import torch.nn as nn
# import torch.optim as optim
# import copy
# from copy import deepcopy
# import matplotlib.pyplot as plt
# # import mymodule as myModules
# import matplotlib.cm as cm
# from scipy import linalg
# from timeit import default_timer as timer
# import networkx as nx
# import os
# import pickle
# import datetime
# #### Import SLOG packakes
# from SLOGmodules import SLOGtools as SLOGtools
# from SLOGmodules import SLOGobjective as SLOGobj
# from SLOGmodules import SLOGarchitectures as SLOGarchi 
# from SLOGmodules import SLOGtraining as SLOGtrainer
# from SLOGmodules import SLOGmodel as SLOGmodel
# from SLOGmodules import SLOGevaluation as SLOGevaluator
# from SLOGmodules import SLOGdata as SLOGdata

# #### Import GNN packages
# from SLOGmodules import graphTools as graphTools
# from SLOGmodules import dataTools as dataTools

# from alegnn.modules import architectures as archit
# from alegnn.modules import model as model
# from alegnn.modules import training as training
# from alegnn.modules import evaluation as evaluation
# from alegnn.modules import loss as loss
# from alegnn.utils.miscTools import writeVarValues
# from alegnn.utils.miscTools import saveSeed

In [2]:
from SLOGmodules import SLOGpipeline as SLOGpipeline
from SLOGmodules import SLOGSaveDir as SLOGSaveDir
import os

In [3]:
### Model parameters
K = 5 # number of layers

# filterTrainType = 'h'
filterTrainType = 'wt'

In [4]:
location = 'home'
# location = 'office'
saveDir_result = SLOGSaveDir.get_save_settings(location)
saveDir_dropbox = saveDir_result['saveDir_dropbox']
saveSettings = saveDir_result['saveSettings']

Parameters

In [5]:
### Simulation parameters
simuParas = {}
nTrain_slog = 200000
batchsize_slog = 400
nValid = batchsize_slog
nTest = batchsize_slog
nEpochs = 50
# model_number = 0
model_number = 1 # SLOG-Net-v3 

### Data parameters
L = 5
alpha = 1.0

### Graph parameters

nNodes = 20 # Number of nodes
nClasses = 4 # Number of classes (i.e. number of communities)
N_C = 2 # Number of sources per signal
C = nNodes # constrain constant
q = 4 # Number of constrain vectors for SLOG-Net-v3 
# nNodes = 100 # Number of nodes
# nClasses = 5 # Number of classes (i.e. number of communities)
# N_C = 10 # Number of sources per signal

simuParas['nNodes'] = nNodes
simuParas['nClasses'] = nClasses
simuParas['N_C'] = N_C
simuParas['S'] = N_C
simuParas['model_number'] = model_number

graphType = 'SBM' # Type of graph
graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
graphOptions['nCommunities'] = nClasses # Number of communities
graphOptions['probIntra'] = 0.8 # Probability of drawing edges intra communities
graphOptions['probInter'] = 0.2 # Probability of drawing edges inter communities

# graphType = 'Random Geometric' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['distance'] = 0.2 # Number of communities

# graphType = 'BA' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['alpha'] = 1.0

# graphType = 'ER' # Type of graph
# graphOptions = {} # Dictionary of options to pass to the graphTools.createGraph function
# graphOptions['probIntra'] = 0.3 # Probability of drawing edges

## Filter type: g or h
# filterType = 'g'
# filterType = 'h'
filterType = 'wt'

## Signal mode: gaussion or 1
signalMode = 'Gaussion'

## Train mode: Wt or default (Vdiag(\tilde{h})V^T)
trainMode = 'default'
# trainMode = 'Wt'

## Filter mode: Wt or default (Vdiag(\tilde{h})V^T)
# filterMode = 'default'
filterMode = 'Wt'

## Selection mode: random or nodes with top-N_C degree
selectMode = 'random'

## Noise level
noiseLevel = 0.0
noiseType = 'uniform'

simuParas['nTrain_slog'] = nTrain_slog
simuParas['batchsize_slog'] = batchsize_slog
simuParas['nValid'] = nValid
simuParas['nTest'] = nTest
simuParas['L'] = L
simuParas['noiseLevel'] = noiseLevel
simuParas['noiseType'] = noiseType
simuParas['filterType'] = filterType
simuParas['signalMode'] = signalMode
simuParas['trainMode'] = trainMode
simuParas['filterMode'] = filterMode
simuParas['selectMode'] = selectMode
simuParas['graphType'] = graphType 
simuParas['alpha'] = alpha
simuParas['nEpochs'] = nEpochs


tMax = None # Maximum number of diffusion times (W^t for t < tMax)
# tMax = nNodes
tMax = 5
simuParas['tMax'] = tMax

## model parameters
modelParas = {}
modelParas['filterTrainType'] = filterTrainType
modelParas['C'] = C
modelParas['q'] = q
modelParas['K'] = K

## Experiment parameters
expParas = {}
expParas['nRealiz'] = 10


In [6]:
# thisFilename_SLOG = 'sourceLocSLOGNET'
# saveDirRoot = 'experiments' # Relative location where to save the file
# saveDir = os.path.join(saveDirRoot, thisFilename_SLOG) # Dir where to save all the results from each run
# saveSettings = {}
# saveSettings['thisFilename_SLOG'] = thisFilename_SLOG         
# saveSettings['saveDirRoot'] = saveDirRoot # Relative location where to save the file
# saveSettings['saveDir'] = os.path.join(saveDirRoot, thisFilename_SLOG) # Dir where to save all the results from each run
# saveDirRoot_dropbox = r"C:\Users\Chang Ye\Dropbox\onlineresults"
# saveSettings['saveDirRoot_dropbox'] = saveDirRoot_dropbox
# saveSettings['saveDir_dropbox'] = os.path.join(saveDirRoot_dropbox, saveDir)

In [7]:
# print(saveSettings['saveDir_dropbox'])

Experiment: noise = 0

In [8]:
simuParas['noiseLevel'] = 0.0
simuParas['model_number'] = 1
modelParas['q'] = 20
modelParas['filterTrainType'] = 'wt'
result_1 = SLOGpipeline.slog_classification_experiments(simuParas = simuParas, 
                 graphOptions = graphOptions, 
                 modelParas = modelParas, expParas = expParas, saveSettings = saveSettings)


Create:  C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-SBM-20220721122029
Device selected: cpu
C =  4 , Community nodes: [10 14 15 16 17 18 19]
Degress of community nodes: [5. 8. 7. 7. 4. 6. 5.]
Sorted degress: [4 0 6 5 2 3 1]
Randomized sorted degress: [4 2 6 1 0 3 5]
C =  4 , Community nodes: [0 1 2 3 4]
Degress of community nodes: [7. 5. 6. 5. 5.]
Sorted degress: [1 3 4 2 0]
Randomized sorted degress: [2 1 3 0 4]
C =  4 , Community nodes: [11 12 13]
Degress of community nodes: [5. 6. 1.]
Sorted degress: [2 0 1]
Randomized sorted degress: [0 2 1]
C =  4 , Community nodes: [5 6 7 8 9]
Degress of community nodes: [8. 6. 7. 4. 7.]
Sorted degress: [3 1 2 4 0]
Randomized sorted degress: [4 3 0 1 2]
Saved to experiments\sourceLocSLOGNET-SBM-20220721122029\cLabels-SBM.npy
Saved to C:\Users\Chang Ye\Dropbox\onlineResults\experiments\sourceLocSLOGNET-SBM-20220721122029\cLabels-SBM.npy
Source nodes: [array([16, 18], dtype=int64), array([0, 4], dtype=int64), array([13, 12


    (E:  1, B: 161) 
	 Loss: 0.0951 [T] 0.0893 [V] 	 RE: 0.9804 [T] 	 RE: 0.9415 [V] 	 Best Loss: 0.0783 [V] 	 Loss: 0.0302 [Test]

    (E:  1, B: 201) 
	 Loss: 0.0849 [T] 0.0946 [V] 	 RE: 0.9559 [T] 	 RE: 0.9690 [V] 	 Best Loss: 0.0783 [V] 	 Loss: 0.0082 [Test]

    (E:  1, B: 241) 
	 Loss: 0.0584 [T] 0.0572 [V] 	 RE: 0.7552 [T] 	 RE: 0.7531 [V] 	 Best Loss: 0.0572 [V] 	 Loss: 0.0036 [Test]
Best model updated at: 0 240
rho_1 tensor([0.0000, 1.1419, 0.0150, 0.4498, 0.0000])
eta_1 tensor([0.0987, 0.0731, 1.3143, 1.5712, 0.9152])
lmbd tensor([0.0884, 0.5215, 0.2670, 0.1398, 0.7528])
alpha_1 tensor([-1.2200, -1.5847,  0.9157, -1.6393, -0.1171])
alpha_2 tensor([-1.4642,  0.7900,  0.2341, -1.4179,  0.0090])
beta_1 tensor([ 1.3185, -1.0368, -1.7701, -0.0547,  0.9721])
beta_2 tensor([-0.2675, -0.3135, -0.9116,  0.7756, -1.0559])
beta_3 tensor([-0.1252, -1.0941,  1.8124,  0.7095,  0.6619])
gamma_1 tensor([-0.7654, -1.9237,  1.9592, -0.8828,  0.8379])
M tensor([[[-1.2812,  1.5732,  0.8348, -1.


    (E:  1, B: 321) 
	 Loss: 0.0029 [T] 0.0003 [V] 	 RE: 0.1744 [T] 	 RE: 0.0561 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0032 [Test]
Best model updated at: 0 320
rho_1 tensor([0.0000, 1.1855, 0.0219, 0.4925, 0.0127])
eta_1 tensor([0.1102, 0.0706, 1.3913, 1.6394, 1.0684])
lmbd tensor([0.0896, 0.5351, 0.2755, 0.2333, 0.7528])
alpha_1 tensor([-1.2199, -1.5804,  0.9147, -1.6742, -0.1171])
alpha_2 tensor([-1.4642,  0.7741,  0.2337, -1.4363,  0.0090])
beta_1 tensor([ 1.3185, -1.0264, -1.7987, -0.0546,  0.9721])
beta_2 tensor([-0.2590, -0.3197, -0.9265,  0.7755, -1.0559])
beta_3 tensor([-0.1265, -1.0961,  1.8123,  0.7092,  0.6619])
gamma_1 tensor([-0.7654, -1.9499,  2.0076, -0.8713,  0.8379])
M tensor([[[-1.1799,  1.7261,  0.9136, -0.9102,  0.5802],
         [-0.0052, -0.6436, -1.1453, -0.4188, -1.9402],
         [ 0.9766, -3.6599,  1.8879,  1.3439,  1.0662],
         ...,
         [-0.2530, -2.3171,  0.9139,  1.9548, -3.4402],
         [-0.0904, -0.9617,  0.0887,  1.0119,  0.2715],
         [


    (E:  4, B: 141) 
	 Loss: 0.0673 [T] 0.0628 [V] 	 RE: 0.8284 [T] 	 RE: 0.7894 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0007 [Test]

    (E:  4, B: 181) 
	 Loss: 0.0433 [T] 0.0353 [V] 	 RE: 0.6383 [T] 	 RE: 0.5916 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0002 [Test]

    (E:  4, B: 221) 
	 Loss: 0.0108 [T] 0.0155 [V] 	 RE: 0.3399 [T] 	 RE: 0.3917 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0002 [Test]

    (E:  4, B: 261) 
	 Loss: 0.0025 [T] 0.0020 [V] 	 RE: 0.1561 [T] 	 RE: 0.1420 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0002 [Test]

    (E:  4, B: 301) 
	 Loss: 0.0023 [T] 0.0013 [V] 	 RE: 0.1540 [T] 	 RE: 0.1137 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0021 [Test]

    (E:  4, B: 341) 
	 Loss: 0.0317 [T] 0.0285 [V] 	 RE: 0.5541 [T] 	 RE: 0.5317 [V] 	 Best Loss: 0.0003 [V] 	 Loss: 0.0020 [Test]


KeyboardInterrupt: 

Experiment: noise = 0.02

In [ ]:
# simuParas['noiseLevel'] = 0.0
# simuParas['model_number'] = 1
# modelParas['q'] = 8
# modelParas['filterTrainType'] = 'h'
# result_2 = SLOGpipeline.slog_experiments(simuParas = simuParas, 
#                  graphOptions = graphOptions, 
#                  modelParas = modelParas, expParas = expParas)



Experiment: noise = 0.05

In [ ]:
# simuParas['noiseLevel'] = 0.05
# result_3 = SLOGpipeline.slog_experiments(simuParas = simuParas, 
#                  graphOptions = graphOptions, 
#                  modelParas = modelParas, expParas = expParas)



In [ ]:
# print(result.get_experiment_result())